In [12]:
import os
import pickle
from gensim.models import KeyedVectors
from gensim.models.wrappers import FastText

In [13]:
from tweet.tweet import Tweet
from preprocessing import preprocessing, tree_create

In [14]:
%%time
modelw2v = KeyedVectors.load_word2vec_format("../model/SBW-vectors-300-min5.bin.gz", binary=True)

CPU times: user 1min 3s, sys: 1.92 s, total: 1min 5s
Wall time: 2min 9s


In [17]:
ROOT_OUT = '../pickle/'

In [18]:
lexicon_attack = ['falso', 'mentira']

## Pipeline

**Primer resultado**

In [83]:
def tagged(tweets, root, lexicon, model, n):
    lexicons = [lexicon]
    tree = tree_create(tweets, root)
    
    for i in range(n):
        if 0 < i:
            new = lexicons[i-1] + [word for word, d in model.most_similar(positive=lexicons[i-1])]
            lexicons.append(new)
       
        for _, tweet in tweets.items():
            tweet.is_attack(lexicons[i])
                   
        name = root + 'n' + str(i) + '.pkl'
        with open(os.path.join(ROOT_OUT, name), 'wb') as fd:
            pickle.dump(tree, fd)
    
    return lexicons

In [87]:
tweets, root = preprocessing('perfil.status.1191780164549697536.json')
lexicons = tagged(tweets, root, lexicon_attack, modelw2v, 3)

IDEA FRANCO:

1- AGRANDO LEXICON A PARTIR DEL MODELO DE CRISTIAN

2- CALCULO UN UMBRAL DEL MODELO DE CRISTIAN

3- PARA CADA (PALABRAL, TOKENT) DEL LEXICON X TWEET, SI DISTANCIA(PALABRAL, TOKENT) > UMBRAL ENTONCES LEXICON + TOKENT 

In [ ]:
from itertools import product

In [ ]:
UMBRAL = 0.72

In [ ]:
for _, tweet in tweets.items():
    merge = product(lexicon_attack, tweet.tokens)
    for wordl, token in list(merge):
        if wordl and token in model.wv.vocab:
            if model.similarity(wordl, token) >= UMBRAL:
                if token not in lexicon_attack:
                    lexicon_attack.append(token)

In [ ]:
lexicon_attack

In [ ]:
w = filter(lambda x: x in modelw2v.vocab, lexicon_attack)
lexicon_attack += [word for word, d in modelw2v.most_similar(positive=w)]

In [ ]:
lexicon_attack

In [ ]:
import nltk.translate.bleu_score as bleu
print('BLEU score: {}'.format(bleu.sentence_bleu(tree.replies[19].tokens, tree.tokens)))

**API TWITTER**

In [ ]:
CONSUMER_KEY = 'tRe3ZBhnXDPBEKyTi4ifvPTC7' 
CONSUMER_SECRET = '0dmQlSimyEmaOPiDoFZTF0wYjO0LSzSdWmCP7afZkNHZF94Af5'
ACCESS_TOKEN = '421837149-YVewRWLSu8xyYNU1H4NtELUejGB8lzvT4KCfallG'
ACCESS_SECRET = 'mFsKYpvJaWvZGHRoy874Z4IF09CRP69srlVj2yjN6mH5L'